In [ ]:
import networkx as nx
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import time
import warnings
import random

# --- V7.0 绘图美学配置 (Aesthetic Setup) ---
warnings.filterwarnings("ignore")
plt.rcParams['font.sans-serif'] = ['Arial', 'SimHei'] # 适配中英文
plt.rcParams['axes.unicode_minus'] = False

try:
    import scienceplots
    plt.style.use(['science', 'no-latex']) # 学术风格
except ImportError:
    # 降级方案：使用 Seaborn 优化
    sns.set_theme(style="whitegrid", palette="deep", context="paper")

class Graph_Solver_Capsule:
    def __init__(self, name="Graph_Model", is_directed=False):
        """
        [MCM Graph Solver V7.0 - Final Version]
        架构: 1.Audit -> 2.Metrics -> 3.Deep Analysis -> 4.Auto-Delivery
        """
        self.name = name
        self.is_directed = is_directed
        self.timestamp = int(time.time())
        self.G = None
        
        # 结果存储容器
        self.node_metrics = pd.DataFrame()
        self.robustness_log = {} # 记录攻击测试的关键结论
        
        # 自动创建输出目录
        self.output_dir = f"./Results_{name}_{self.timestamp}"
        if not os.path.exists(self.output_dir):
            os.makedirs(self.output_dir)

    # ======================================================
    # Phase 0: 握手与协议 (Setup & Handshake)
    # ======================================================
    def generate_handshake(self, df_dict=None):
        print(f"\n🤝 === 复制以下 Prompt 发送给 AI (V7.0 Graph) ===\n")
        print(f"【系统设定】\n我正在使用 `Graph_Solver_Capsule` (V7.0)。")
        print(f"图类型: {'有向图 (DiGraph)' if self.is_directed else '无向图 (Graph)'}。")
        print(f"输出目录: `{self.output_dir}`")
        
        print("\n【API 接口清单】")
        print("1. 构建: solver.build_from_edgelist(df, source='S', target='T', weight='W')")
        print("2. 自检: solver.audit() # 必须执行，逻辑自检")
        print("3. 指标: solver.compute_centrality()")
        print("4. [核武器] 社团: solver.detect_communities(method='louvain')")
        print("5. [核武器] 鲁棒性: solver.analyze_robustness(attack_type='targeted', fraction=0.3)")
        print("6. [可视化] 绘图: solver.plot_network(layout='kamada_kawai')")
        print("7. [交付] 导出: solver.export_results() # 生成全套交付物")
        
        print("\n【⚠️ 必须注意的图论陷阱】")
        print("1. **负权边 (Negative Weights)**: 如存在，Dijkstra 失效，请提示使用 Bellman-Ford。")
        print("2. **连通性 (Connectivity)**: 若图不连通，Global Efficiency 无物理意义。建议提取 LCC (Largest Connected Component)。")
        print("3. **强/弱连通**: 有向图中请区分 Strong (双向可达) vs Weak (忽略方向)。")
        
        if df_dict:
            print("\n【数据摘要】")
            for name, df in df_dict.items():
                print(f"Dataset '{name}': {list(df.columns)} | Rows: {len(df)}")

    # ======================================================
    # Phase 1: 构建与自检 (Build & Audit)
    # ======================================================
    def build_from_edgelist(self, df, source, target, weight=None):
        print(f"\n🏗️ 正在构建网络...")
        create_using = nx.DiGraph() if self.is_directed else nx.Graph()
        
        try:
            # 自动处理列名空格问题
            self.G = nx.from_pandas_edgelist(df, source=source, target=target, edge_attr=weight, create_using=create_using)
        except Exception as e:
            print(f"❌ 构建失败: {e}")
            return

        print(f"✅ 网络构建完成。Nodes: {self.G.number_of_nodes()}, Edges: {self.G.number_of_edges()}")
        # 初始化指标表
        self.node_metrics = pd.DataFrame(index=list(self.G.nodes()))
        self.node_metrics.index.name = 'Node_ID'

    def audit(self):
        print("\n🛡️ === 逻辑审计 (Network Audit) =====")
        if self.G is None: raise ValueError("❌ 图未构建。")
        
        # 1. 连通性检查
        if self.is_directed:
            n_comp = nx.number_strongly_connected_components(self.G)
            n_weak = nx.number_weakly_connected_components(self.G)
            print(f"info: 强连通分量: {n_comp}, 弱连通分量: {n_weak}")
            if n_weak > 1: print("⚠️ 警告: 网络不是(弱)连通的！")
        else:
            n_comp = nx.number_connected_components(self.G)
            print(f"info: 连通分量: {n_comp}")
            if n_comp > 1:
                largest = len(max(nx.connected_components(self.G), key=len))
                ratio = largest / len(self.G)
                print(f"⚠️ 警告: 网络不连通！最大连通子图(LCC)占比: {ratio:.1%}")
                print("💡 建议: 在论文中声明“后续分析基于最大连通子图(LCC)”。")

        # 2. 孤立点
        isolates = list(nx.isolates(self.G))
        if isolates:
            print(f"⚠️ 警告: 发现 {len(isolates)} 个孤立点 (Isolates)。建议预处理移除。")
            
        # 3. 自环
        loops = list(nx.selfloop_edges(self.G))
        if loops:
            print(f"⚠️ 警告: 发现 {len(loops)} 条自环。")
        
        print("✅ 审计结束。")

    # ======================================================
    # Phase 2: 基础指标 (Metrics)
    # ======================================================
    def compute_centrality(self):
        print(f"\n📐 计算中心度指标...")
        
        # Degree
        if self.is_directed:
            self.node_metrics['In_Degree'] = pd.Series(nx.in_degree_centrality(self.G))
            self.node_metrics['Out_Degree'] = pd.Series(nx.out_degree_centrality(self.G))
        else:
            self.node_metrics['Degree'] = pd.Series(nx.degree_centrality(self.G))
            
        # Betweenness (大图优化：采样)
        if len(self.G) > 2000:
            print("⚠️ 警告: 节点数>2000，启用 k=100 采样加速计算 Betweenness。")
            bet = nx.betweenness_centrality(self.G, k=100)
        else:
            bet = nx.betweenness_centrality(self.G)
        self.node_metrics['Betweenness'] = pd.Series(bet)
        
        # Closeness
        self.node_metrics['Closeness'] = pd.Series(nx.closeness_centrality(self.G))
        
        # Eigenvector (处理不收敛)
        try:
            eig = nx.eigenvector_centrality(self.G, max_iter=600)
            self.node_metrics['Eigenvector'] = pd.Series(eig)
        except:
            print("⚠️ Eigenvector 未收敛，填充 0。")
            self.node_metrics['Eigenvector'] = 0.0

        print("✅ 基础指标已存入 self.node_metrics。")

    # ======================================================
    # Phase 3: 深度分析 (Deep Analysis - O奖核武器)
    # ======================================================
    def detect_communities(self, method='louvain'):
        print(f"\n🧩 启动社团检测 (Method: {method})...")
        if method == 'louvain':
            try:
                import community.community_louvain as community_louvain
                # Louvain 需转无向
                g_temp = self.G.to_undirected() if self.is_directed else self.G
                partition = community_louvain.best_partition(g_temp)
                modularity = community_louvain.modularity(partition, g_temp)
                
                self.node_metrics['Community_ID'] = self.node_metrics.index.map(partition)
                print(f"✅ 社团划分完成 (Louvain)。Modularity Q={modularity:.4f}")
                print(f"   共发现 {len(set(partition.values()))} 个社团。")
                return modularity
            except ImportError:
                print("❌ 未安装 `python-louvain`。请 pip install python-louvain。")
        else:
            print("⚠️ 目前仅封装了 Louvain 算法。")

    def analyze_robustness(self, attack_type='targeted', fraction=0.3):
        """
        [核武器] 鲁棒性分析
        输出：Robustness_Curve.svg
        """
        print(f"\n💣 启动鲁棒性分析 (Type: {attack_type}, Remove: {fraction*100:.0f}%)...")
        
        # 1. 准备副本 (转无向以分析连通性)
        G_sim = self.G.copy()
        if self.is_directed: G_sim = G_sim.to_undirected()
        
        N = G_sim.number_of_nodes()
        remove_count = int(N * fraction)
        
        # 2. 确定移除列表
        if attack_type == 'targeted':
            # 按度降序 (蓄意攻击)
            nodes_sorted = sorted(G_sim.degree, key=lambda x: x[1], reverse=True)
            targets = [n[0] for n in nodes_sorted]
        else:
            # 随机乱序 (随机故障)
            targets = list(G_sim.nodes())
            random.shuffle(targets)
            
        targets = targets[:remove_count]
        
        # 3. 模拟攻击 (使用 np.linspace 减少绘图点数，提高效率)
        steps = np.unique(np.linspace(0, remove_count, 25, dtype=int))
        
        history = {'Removed_Pct': [], 'Giant_Component': [], 'Efficiency': []}
        
        # 计算基准效率 (大图跳过)
        calc_eff = N < 1500
        base_eff = nx.global_efficiency(G_sim) if calc_eff else 1.0
        
        current_removed = 0
        for target_count in steps:
            # 移除增量
            to_remove = target_count - current_removed
            if to_remove > 0:
                nodes_to_drop = targets[current_removed : target_count]
                G_sim.remove_nodes_from(nodes_to_drop)
                current_removed = target_count
            
            # 记录 X 轴
            history['Removed_Pct'].append(current_removed / N)
            
            # 记录 Y1: 最大连通子图占比
            if len(G_sim) > 0:
                lc = len(max(nx.connected_components(G_sim), key=len))
                history['Giant_Component'].append(lc / N)
            else:
                history['Giant_Component'].append(0)
                
            # 记录 Y2: 全局效率 (相对值)
            if calc_eff:
                eff = nx.global_efficiency(G_sim)
                history['Efficiency'].append(eff / base_eff)
            else:
                history['Efficiency'].append(0)
                
        # 4. 绘图
        df_res = pd.DataFrame(history)
        plt.figure(figsize=(8, 5))
        plt.plot(df_res['Removed_Pct'], df_res['Giant_Component'], 'o-', label='Giant Component Size', color='#d62728', lw=2)
        if calc_eff:
            plt.plot(df_res['Removed_Pct'], df_res['Efficiency'], 's--', label='Global Efficiency', color='#1f77b4', lw=2)
            
        plt.title(f"Network Robustness: {attack_type.title()} Attack")
        plt.xlabel("Fraction of Nodes Removed")
        plt.ylabel("Relative Metric Score")
        plt.legend()
        plt.grid(True, alpha=0.3)
        plt.savefig(f"{self.output_dir}/Robustness_Curve_{attack_type}.svg", dpi=300)
        plt.show()
        
        # 记录结论
        collapse_val = df_res[df_res['Giant_Component'] < 0.5].iloc[0]['Removed_Pct'] if (df_res['Giant_Component'] < 0.5).any() else 1.0
        self.robustness_log[attack_type] = collapse_val
        print(f"✅ 鲁棒性分析完成。崩溃点 (GC<0.5) 约在移除 {collapse_val:.1%} 节点时。")

    # ======================================================
    # Phase 4: 顶级可视化 (Visualization)
    # ======================================================
    def plot_network(self, layout='kamada_kawai'):
        print(f"\n🎨 绘制网络图 (Layout: {layout})...")
        plt.figure(figsize=(12, 12))
        
        # 1. 布局安全降级逻辑 (防止 Kamada-Kawai 卡死)
        pos = None
        if layout == 'kamada_kawai':
            if len(self.G) > 1000:
                print("⚠️ 节点数 > 1000，强制降级为 spring_layout 以防卡死。")
                pos = nx.spring_layout(self.G, seed=42)
            else:
                try:
                    pos = nx.kamada_kawai_layout(self.G)
                except:
                    pos = nx.spring_layout(self.G, seed=42)
        else:
            pos = nx.spring_layout(self.G, seed=42)
            
        # 2. 样式映射
        deg = dict(self.G.degree())
        # 大小归一化
        sizes = [v * 5 + 30 for v in deg.values()] 
        
        # 颜色映射 (社团 or 默认)
        if 'Community_ID' in self.node_metrics.columns:
            colors = self.node_metrics.loc[list(self.G.nodes()), 'Community_ID']
            cmap = 'tab20'
        else:
            colors = '#69b3a2'
            cmap = None
            
        # 3. 绘制
        nx.draw_networkx_nodes(self.G, pos, node_size=sizes, node_color=colors, cmap=cmap, alpha=0.9, edgecolors='white')
        nx.draw_networkx_edges(self.G, pos, alpha=0.2, edge_color='gray')
        
        # 4. 智能标签 (仅 Top 10)
        top_nodes = sorted(deg.items(), key=lambda x: x[1], reverse=True)[:10]
        for node, val in top_nodes:
            x, y = pos[node]
            plt.text(x, y+0.03, str(node), fontsize=11, fontweight='bold', ha='center', 
                     bbox=dict(facecolor='white', alpha=0.8, edgecolor='none', pad=1))

        plt.title(f"Network Visualization ({self.name})", fontsize=16)
        plt.axis('off')
        plt.savefig(f"{self.output_dir}/Network_Viz.svg", dpi=300, bbox_inches='tight')
        plt.show()

    def export_to_gephi(self):
        try:
            # 属性注入 (类型转换防止报错)
            for col in self.node_metrics.columns:
                attrs = {k: float(v) if isinstance(v, float) else int(v) 
                         for k, v in self.node_metrics[col].to_dict().items()}
                nx.set_node_attributes(self.G, attrs, col)
            nx.write_gexf(self.G, f"{self.output_dir}/{self.name}.gexf")
            print("✅ Gephi 文件已生成 (.gexf)。")
        except Exception as e:
            print(f"❌ Gephi 导出跳过: {e}")

    # ======================================================
    # Phase 5: 交付 (Delivery)
    # ======================================================
    def export_results(self):
        print(f"\n📦 === 生成交付物至 {self.output_dir} === ")
        
        # 1. Excel
        self.node_metrics.to_excel(f"{self.output_dir}/Node_Metrics.xlsx")
        
        # 2. Gephi
        self.export_to_gephi()
        
        # 3. 智能战报 (Auto-Report.md)
        report = f"# V7.0 Graph Analysis Report: {self.name}\n\n"
        
        report += "## 1. Network Summary\n"
        report += f"- **Type**: {'Directed' if self.is_directed else 'Undirected'}\n"
        report += f"- **Nodes**: {self.G.number_of_nodes()}, **Edges**: {self.G.number_of_edges()}\n"
        report += f"- **Density**: {nx.density(self.G):.4f}\n\n"
        
        report += "## 2. Key Insights\n"
        if not self.node_metrics.empty:
            # 找出指标最高的节点
            col = 'Degree' if 'Degree' in self.node_metrics else 'In_Degree'
            top_deg = self.node_metrics[col].idxmax()
            report += f"* **Hub Node**: Node `{top_deg}` has the highest degree centrality.\n"
            
            if 'Betweenness' in self.node_metrics:
                top_bet = self.node_metrics['Betweenness'].idxmax()
                report += f"* **Bridge Node**: Node `{top_bet}` has the highest betweenness, controlling flow.\n"
                
            if 'Community_ID' in self.node_metrics.columns:
                n_comm = self.node_metrics['Community_ID'].nunique()
                report += f"* **Community Structure**: The network divides into **{n_comm}** communities (Louvain).\n"
        
        if self.robustness_log:
            report += "\n## 3. Resilience Analysis (Robustness)\n"
            for atk, val in self.robustness_log.items():
                status = "Fragile" if val < 0.1 else ("Robust" if val > 0.4 else "Moderate")
                report += f"* Under **{atk} attack**, the network collapses (Giant Component < 50%) after removing **{val:.1%}** of nodes. Rating: **{status}**.\n"
        
        with open(f"{self.output_dir}/Report.md", "w", encoding="utf-8") as f:
            f.write(report)
            
        print(f"✅ [1] 数据表: Node_Metrics.xlsx")
        print(f"✅ [2] 矢量图: Network_Viz.svg, Robustness_*.svg")
        print(f"✅ [3] 交互文件: {self.name}.gexf")
        print(f"✅ [4] 智能战报: Report.md (含自动结论)")